# Data Preprocessing and Feature Engineering

Objective:
- Clean and handle missing values
- Fix data quality issues identified in EDA
- Create new features for modeling
- Encode categorical variables
- Prepare data for machine learning models

Key Issues to Address:
- Column name typo: coustomer_key -> customer_key
- Missing values: 3,723 in fact_sales.unit, 27 in customer names, 1 in item unit
- Date parsing and time-based feature creation
- RFM scores and customer segmentation features
- Product category encoding

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler
from datetime import datetime
import warnings

pd.set_option('display.max_columns', None)
warnings.filterwarnings('ignore')

data_path = 'data/'

fact_sales = pd.read_csv(f'{data_path}fact_sales.csv', encoding='latin-1')
dim_customer = pd.read_csv(f'{data_path}dim_customer.csv', encoding='latin-1')
dim_item = pd.read_csv(f'{data_path}dim_item.csv', encoding='latin-1')
dim_store = pd.read_csv(f'{data_path}dim_store.csv', encoding='latin-1')
dim_time = pd.read_csv(f'{data_path}dim_time.csv', encoding='latin-1')
dim_payment = pd.read_csv(f'{data_path}dim_payment.csv', encoding='latin-1')

print("Data loaded successfully")
print(f"Starting preprocessing for {fact_sales.shape[0]:,} transactions")

Data loaded successfully
Starting preprocessing for 1,000,000 transactions


## Data Quality Fixes

Addressing column naming issues and missing values identified during EDA.

In [2]:
print("FIXING COLUMN NAME ISSUES:")
print("="*80)

print("\nBefore:")
print(f"Fact Sales columns with 'coustomer': {[col for col in fact_sales.columns if 'coustomer' in col.lower()]}")
print(f"Customer dim columns with 'coustomer': {[col for col in dim_customer.columns if 'coustomer' in col.lower()]}")

fact_sales.rename(columns={'coustomer_key': 'customer_key'}, inplace=True)
dim_customer.rename(columns={'coustomer_key': 'customer_key'}, inplace=True)

print("\nAfter:")
print(f"Fact Sales columns: {fact_sales.columns.tolist()}")
print(f"Customer dim columns: {dim_customer.columns.tolist()}")

print("\nColumn name corrections applied successfully")

FIXING COLUMN NAME ISSUES:

Before:
Fact Sales columns with 'coustomer': ['coustomer_key']
Customer dim columns with 'coustomer': ['coustomer_key']

After:
Fact Sales columns: ['payment_key', 'customer_key', 'time_key', 'item_key', 'store_key', 'quantity', 'unit', 'unit_price', 'total_price']
Customer dim columns: ['customer_key', 'name', 'contact_no', 'nid']

Column name corrections applied successfully


In [3]:
print("MISSING VALUE ANALYSIS:")
print("="*80)

print("\nFact Sales missing values:")
print(fact_sales.isnull().sum())

print("\nCustomer dimension missing values:")
print(dim_customer.isnull().sum())

print("\nItem dimension missing values:")
print(dim_item.isnull().sum())

print("\n" + "="*80)
print("HANDLING MISSING VALUES:")
print("="*80)

print("\n1. Fact Sales - Unit column (3,723 missing):")
fact_sales_with_item = fact_sales.merge(dim_item[['item_key', 'unit']], 
                                         on='item_key', 
                                         how='left', 
                                         suffixes=('', '_from_item'))
fact_sales['unit'] = fact_sales['unit'].fillna(fact_sales_with_item['unit_from_item'])
remaining_missing = fact_sales['unit'].isnull().sum()
print(f"   After filling from item dimension: {remaining_missing} missing")

if remaining_missing > 0:
    mode_unit = fact_sales['unit'].mode()[0]
    fact_sales['unit'].fillna(mode_unit, inplace=True)
    print(f"   Filled remaining with mode: '{mode_unit}'")

print(f"\n2. Customer dimension - Name column (27 missing):")
dim_customer['name'].fillna('Unknown', inplace=True)
print(f"   Filled with 'Unknown'")

print(f"\n3. Item dimension - Unit column (1 missing):")
mode_item_unit = dim_item['unit'].mode()[0]
dim_item['unit'].fillna(mode_item_unit, inplace=True)
print(f"   Filled with mode: '{mode_item_unit}'")

print(f"\n4. Payment dimension - Bank name (1 missing for cash):")
dim_payment['bank_name'].fillna('N/A', inplace=True)
print(f"   Filled with 'N/A' for cash transactions")

print("\n" + "="*80)
print("VERIFICATION - Remaining missing values:")
print(f"Fact Sales: {fact_sales.isnull().sum().sum()}")
print(f"Customer: {dim_customer.isnull().sum().sum()}")
print(f"Item: {dim_item.isnull().sum().sum()}")
print(f"Store: {dim_store.isnull().sum().sum()}")
print(f"Time: {dim_time.isnull().sum().sum()}")
print(f"Payment: {dim_payment.isnull().sum().sum()}")

MISSING VALUE ANALYSIS:

Fact Sales missing values:
payment_key        0
customer_key       0
time_key           0
item_key           0
store_key          0
quantity           0
unit            3723
unit_price         0
total_price        0
dtype: int64

Customer dimension missing values:
customer_key     0
name            27
contact_no       0
nid              0
dtype: int64

Item dimension missing values:
item_key       0
item_name      0
desc           0
unit_price     0
man_country    0
supplier       0
unit           1
dtype: int64

HANDLING MISSING VALUES:

1. Fact Sales - Unit column (3,723 missing):
   After filling from item dimension: 3723 missing
   Filled remaining with mode: 'ct'

2. Customer dimension - Name column (27 missing):
   Filled with 'Unknown'

3. Item dimension - Unit column (1 missing):
   Filled with mode: 'ct'

4. Payment dimension - Bank name (1 missing for cash):
   Filled with 'N/A' for cash transactions

VERIFICATION - Remaining missing values:
Fact Sale

## Master Dataset Creation

Creating a comprehensive denormalized dataset with all corrections and joins for feature engineering.`

In [4]:
master_df = fact_sales.copy()

master_df = master_df.merge(dim_customer, on='customer_key', how='left', suffixes=('', '_cust'))
master_df = master_df.merge(dim_item, on='item_key', how='left', suffixes=('', '_item'))
master_df = master_df.merge(dim_store, on='store_key', how='left')
master_df = master_df.merge(dim_time, on='time_key', how='left')
master_df = master_df.merge(dim_payment, on='payment_key', how='left')

master_df['date_parsed'] = pd.to_datetime(master_df['date'], format='%d-%m-%Y %H:%M')

print("Master dataset created with all dimensions")
print(f"Shape: {master_df.shape[0]:,} rows x {master_df.shape[1]} columns")
print(f"\nMissing values in master dataset: {master_df.isnull().sum().sum()}")
print(f"\nColumns: {master_df.columns.tolist()}")

Master dataset created with all dimensions
Shape: 1,000,000 rows x 31 columns

Missing values in master dataset: 0

Columns: ['payment_key', 'customer_key', 'time_key', 'item_key', 'store_key', 'quantity', 'unit', 'unit_price', 'total_price', 'name', 'contact_no', 'nid', 'item_name', 'desc', 'unit_price_item', 'man_country', 'supplier', 'unit_item', 'division', 'district', 'upazila', 'date', 'hour', 'day', 'week', 'month', 'quarter', 'year', 'trans_type', 'bank_name', 'date_parsed']


## Feature Engineering

Creating new features to enhance model performance:
- Temporal features (day of week, time of day categories, etc.)
- Customer behavior features (RFM scores, purchase patterns)
- Product features (price categories, popularity metrics)
- Geographic features
- Interaction features

In [5]:
print("CREATING TEMPORAL FEATURES:")
print("="*80)

master_df['day_of_week'] = master_df['date_parsed'].dt.dayofweek
master_df['day_name'] = master_df['date_parsed'].dt.day_name()
master_df['is_weekend'] = master_df['day_of_week'].isin([5, 6]).astype(int)
master_df['week_of_year'] = master_df['date_parsed'].dt.isocalendar().week

def categorize_time_of_day(hour):
    if 6 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 17:
        return 'Afternoon'
    elif 17 <= hour < 21:
        return 'Evening'
    else:
        return 'Night'

master_df['time_of_day'] = master_df['hour'].apply(categorize_time_of_day)

master_df['is_peak_hour'] = master_df['hour'].isin([9, 13, 14, 17, 19]).astype(int)

master_df['year_month'] = master_df['date_parsed'].dt.to_period('M')
master_df['days_since_start'] = (master_df['date_parsed'] - master_df['date_parsed'].min()).dt.days

print("Temporal features created:")
print("- day_of_week, day_name")
print("- is_weekend")
print("- week_of_year")
print("- time_of_day (Morning/Afternoon/Evening/Night)")
print("- is_peak_hour")
print("- year_month")
print("- days_since_start")

print(f"\nSample of new temporal features:")
print(master_df[['date_parsed', 'day_of_week', 'day_name', 'is_weekend', 
                 'time_of_day', 'is_peak_hour']].head(10))

CREATING TEMPORAL FEATURES:
Temporal features created:
- day_of_week, day_name
- is_weekend
- week_of_year
- time_of_day (Morning/Afternoon/Evening/Night)
- is_peak_hour
- year_month
- days_since_start

Sample of new temporal features:
          date_parsed  day_of_week  day_name  is_weekend time_of_day  \
0 2016-04-26 17:13:00            1   Tuesday           0     Evening   
1 2018-06-11 19:08:00            0    Monday           0     Evening   
2 2016-11-19 08:19:00            5  Saturday           1     Morning   
3 2020-02-01 06:00:00            5  Saturday           1     Morning   
4 2014-07-15 02:58:00            1   Tuesday           0       Night   
5 2019-06-25 15:36:00            1   Tuesday           0   Afternoon   
6 2019-12-17 01:12:00            1   Tuesday           0       Night   
7 2017-11-16 07:07:00            3  Thursday           0     Morning   
8 2016-10-16 05:49:00            6    Sunday           1       Night   
9 2018-02-13 01:12:00            1   Tuesday

In [6]:
print("CREATING CUSTOMER BEHAVIOR FEATURES:")
print("="*80)

reference_date = master_df['date_parsed'].max()

customer_features = master_df.groupby('customer_key').agg({
    'date_parsed': lambda x: (reference_date - x.max()).days,
    'payment_key': 'count',
    'total_price': ['sum', 'mean', 'std'],
    'quantity': ['sum', 'mean'],
    'item_key': 'nunique',
    'store_key': 'nunique',
    'division': lambda x: x.mode()[0] if len(x) > 0 else 'Unknown'
}).reset_index()

customer_features.columns = ['customer_key', 'recency', 'frequency', 'monetary_total', 
                            'monetary_avg', 'monetary_std', 'total_quantity', 
                            'avg_quantity', 'unique_items', 'unique_stores', 
                            'preferred_division']

customer_features['monetary_std'].fillna(0, inplace=True)

customer_features['r_score'] = pd.qcut(customer_features['recency'], q=4, 
                                        labels=[4, 3, 2, 1], duplicates='drop')
customer_features['f_score'] = pd.qcut(customer_features['frequency'].rank(method='first'), 
                                        q=4, labels=[1, 2, 3, 4], duplicates='drop')
customer_features['m_score'] = pd.qcut(customer_features['monetary_total'], q=4, 
                                        labels=[1, 2, 3, 4], duplicates='drop')

customer_features['rfm_score'] = (customer_features['r_score'].astype(int) + 
                                  customer_features['f_score'].astype(int) + 
                                  customer_features['m_score'].astype(int))

def segment_customer(row):
    if row['rfm_score'] >= 10:
        return 'Champions'
    elif row['rfm_score'] >= 8:
        return 'Loyal Customers'
    elif row['rfm_score'] >= 6:
        return 'Potential Loyalists'
    elif row['rfm_score'] >= 5:
        return 'At Risk'
    else:
        return 'Lost Customers'

customer_features['customer_segment'] = customer_features.apply(segment_customer, axis=1)

master_df = master_df.merge(customer_features[['customer_key', 'recency', 'frequency', 
                                               'monetary_avg', 'rfm_score', 'customer_segment',
                                               'unique_items', 'unique_stores']], 
                            on='customer_key', how='left')

print("Customer behavior features created:")
print("- recency (days since last purchase)")
print("- frequency (total purchases)")
print("- monetary_avg (average order value)")
print("- rfm_score (combined RFM score)")
print("- customer_segment (Champions/Loyal/Potential/At Risk/Lost)")
print("- unique_items (product diversity)")
print("- unique_stores (store diversity)")

print(f"\nCustomer segment distribution in master dataset:")
print(master_df['customer_segment'].value_counts())

CREATING CUSTOMER BEHAVIOR FEATURES:
Customer behavior features created:
- recency (days since last purchase)
- frequency (total purchases)
- monetary_avg (average order value)
- rfm_score (combined RFM score)
- customer_segment (Champions/Loyal/Potential/At Risk/Lost)
- unique_items (product diversity)
- unique_stores (store diversity)

Customer segment distribution in master dataset:
customer_segment
Loyal Customers        281267
Potential Loyalists    263579
Champions              259003
Lost Customers         108412
At Risk                 87739
Name: count, dtype: int64


In [7]:
print("CREATING PRODUCT FEATURES:")
print("="*80)

product_popularity = master_df.groupby('item_key').agg({
    'payment_key': 'count',
    'total_price': 'sum',
    'quantity': 'sum'
}).reset_index()

product_popularity.columns = ['item_key', 'product_transaction_count', 
                              'product_total_revenue', 'product_total_units']

product_popularity['product_popularity_rank'] = product_popularity['product_transaction_count'].rank(
    ascending=False, method='dense'
).astype(int)

master_df = master_df.merge(product_popularity, on='item_key', how='left')

def categorize_price(price):
    if price < 10:
        return 'Budget'
    elif price < 20:
        return 'Standard'
    elif price < 30:
        return 'Premium'
    else:
        return 'Luxury'

master_df['price_category'] = master_df['unit_price'].apply(categorize_price)

master_df['discount_indicator'] = (master_df['unit_price'] < master_df['unit_price_item']).astype(int)

category_avg_price = master_df.groupby('desc')['unit_price'].mean()
master_df['category_avg_price'] = master_df['desc'].map(category_avg_price)
master_df['price_vs_category_avg'] = master_df['unit_price'] / master_df['category_avg_price']

master_df['revenue_per_unit'] = master_df['total_price'] / master_df['quantity']

print("Product features created:")
print("- product_transaction_count (popularity)")
print("- product_total_revenue")
print("- product_popularity_rank")
print("- price_category (Budget/Standard/Premium/Luxury)")
print("- discount_indicator")
print("- price_vs_category_avg (relative pricing)")
print("- revenue_per_unit")

print(f"\nPrice category distribution:")
print(master_df['price_category'].value_counts())

print(f"\nSample of product features:")
print(master_df[['item_name', 'unit_price', 'price_category', 
                 'product_popularity_rank', 'discount_indicator']].head(10))

CREATING PRODUCT FEATURES:
Product features created:
- product_transaction_count (popularity)
- product_total_revenue
- product_popularity_rank
- price_category (Budget/Standard/Premium/Luxury)
- discount_indicator
- price_vs_category_avg (relative pricing)
- revenue_per_unit

Price category distribution:
price_category
Standard    616996
Premium     178278
Budget      128903
Luxury       75823
Name: count, dtype: int64

Sample of product features:
                            item_name  unit_price price_category  \
0             M&M Peanut Candy 1.7 oz       35.00         Luxury   
1           Charmin Ultra Bath Tissue       26.00        Premium   
2       Dole Fruit in Gel Cups 4.3 oz       12.50       Standard   
3      Paper Bowls 20 oz Ultra Strong       14.00       Standard   
4  Waterloo Sparkling Watermelon 12oz        8.00         Budget   
5    Premier Protein Shake Choc. 11oz       22.00        Premium   
6   Brisk Lemon Iced Tea - 12 oz cans       15.50       Standard   
7  

In [8]:
print("CREATING GEOGRAPHIC AND INTERACTION FEATURES:")
print("="*80)

store_performance = master_df.groupby('store_key').agg({
    'total_price': ['sum', 'count', 'mean']
}).reset_index()
store_performance.columns = ['store_key', 'store_total_revenue', 
                             'store_transaction_count', 'store_avg_transaction']

master_df = master_df.merge(store_performance, on='store_key', how='left')

division_performance = master_df.groupby('division').agg({
    'total_price': 'mean',
    'payment_key': 'count'
}).reset_index()
division_performance.columns = ['division', 'division_avg_transaction', 'division_transaction_count']

master_df = master_df.merge(division_performance, on='division', how='left')

master_df['is_dhaka'] = (master_df['division'] == 'DHAKA').astype(int)
master_df['is_top_division'] = master_df['division'].isin(['DHAKA', 'CHITTAGONG', 'RAJSHAHI']).astype(int)

master_df['customer_product_interaction'] = master_df['frequency'] * master_df['product_transaction_count']
master_df['customer_spending_velocity'] = master_df['monetary_avg'] / (master_df['recency'] + 1)
master_df['product_quantity_ratio'] = master_df['quantity'] / master_df['product_total_units']

print("Geographic features created:")
print("- store_total_revenue, store_transaction_count")
print("- division_avg_transaction")
print("- is_dhaka, is_top_division")

print("\nInteraction features created:")
print("- customer_product_interaction")
print("- customer_spending_velocity")
print("- product_quantity_ratio")

print(f"\nDhaka vs Other divisions:")
print(master_df['is_dhaka'].value_counts())

print(f"\nCurrent master dataset shape: {master_df.shape}")
print(f"Total features: {master_df.shape[1]}")

CREATING GEOGRAPHIC AND INTERACTION FEATURES:
Geographic features created:
- store_total_revenue, store_transaction_count
- division_avg_transaction
- is_dhaka, is_top_division

Interaction features created:
- customer_product_interaction
- customer_spending_velocity
- product_quantity_ratio

Dhaka vs Other divisions:
is_dhaka
0    613112
1    386888
Name: count, dtype: int64

Current master dataset shape: (1000000, 65)
Total features: 65


## Categorical Variable Encoding

Preparing categorical variables for machine learning models through label encoding and one-hot encoding.

In [9]:
print("ENCODING CATEGORICAL VARIABLES:")
print("="*80)

categorical_cols = ['desc', 'division', 'district', 'quarter', 'trans_type', 
                   'time_of_day', 'day_name', 'customer_segment', 'price_category']

label_encoders = {}

for col in categorical_cols:
    if col in master_df.columns:
        le = LabelEncoder()
        master_df[f'{col}_encoded'] = le.fit_transform(master_df[col].astype(str))
        label_encoders[col] = le
        print(f"- {col}: {len(le.classes_)} unique values encoded")

print("\n" + "="*80)
print("CREATING ONE-HOT ENCODED FEATURES FOR KEY CATEGORIES:")

payment_dummies = pd.get_dummies(master_df['trans_type'], prefix='payment')
time_dummies = pd.get_dummies(master_df['time_of_day'], prefix='time')
segment_dummies = pd.get_dummies(master_df['customer_segment'], prefix='segment')

master_df = pd.concat([master_df, payment_dummies, time_dummies, segment_dummies], axis=1)

print(f"\nOne-hot encoded columns added:")
print(f"- Payment types: {list(payment_dummies.columns)}")
print(f"- Time of day: {list(time_dummies.columns)}")
print(f"- Customer segments: {list(segment_dummies.columns)}")

print(f"\nFinal master dataset shape: {master_df.shape}")
print(f"Total columns: {master_df.shape[1]}")

ENCODING CATEGORICAL VARIABLES:
- desc: 30 unique values encoded
- division: 7 unique values encoded
- district: 64 unique values encoded
- quarter: 4 unique values encoded
- trans_type: 3 unique values encoded
- time_of_day: 4 unique values encoded
- day_name: 7 unique values encoded
- customer_segment: 5 unique values encoded
- price_category: 4 unique values encoded

CREATING ONE-HOT ENCODED FEATURES FOR KEY CATEGORIES:

One-hot encoded columns added:
- Payment types: ['payment_card', 'payment_cash', 'payment_mobile']
- Time of day: ['time_Afternoon', 'time_Evening', 'time_Morning', 'time_Night']
- Customer segments: ['segment_At Risk', 'segment_Champions', 'segment_Lost Customers', 'segment_Loyal Customers', 'segment_Potential Loyalists']

Final master dataset shape: (1000000, 86)
Total columns: 86


In [10]:
print("PREPROCESSING SUMMARY:")
print("="*80)

print("\n1. DATA QUALITY FIXES:")
print("   - Fixed column name: coustomer_key -> customer_key")
print("   - Handled all missing values (0 remaining)")
print("   - Cleaned and standardized all dimension tables")

print("\n2. FEATURE ENGINEERING SUMMARY:")
print("   Original columns: 31")
print("   Total columns after engineering: 86")
print("   New features created: 55")

print("\n3. FEATURE CATEGORIES:")
feature_categories = {
    'Temporal Features': ['day_of_week', 'is_weekend', 'time_of_day', 'is_peak_hour', 
                         'days_since_start', 'week_of_year'],
    'Customer Features': ['recency', 'frequency', 'monetary_avg', 'rfm_score', 
                         'customer_segment', 'unique_items', 'unique_stores'],
    'Product Features': ['product_transaction_count', 'product_popularity_rank', 
                        'price_category', 'price_vs_category_avg', 'discount_indicator'],
    'Geographic Features': ['store_total_revenue', 'division_avg_transaction', 
                           'is_dhaka', 'is_top_division'],
    'Interaction Features': ['customer_product_interaction', 'customer_spending_velocity', 
                            'product_quantity_ratio'],
    'Encoded Features': ['desc_encoded', 'division_encoded', 'trans_type_encoded', 
                        'customer_segment_encoded'],
    'One-Hot Features': ['payment_card', 'payment_cash', 'payment_mobile', 
                        'time_Morning', 'time_Afternoon', 'time_Evening', 'time_Night']
}

for category, features in feature_categories.items():
    available = [f for f in features if f in master_df.columns]
    print(f"\n   {category}: {len(available)} features")
    print(f"      {', '.join(available[:5])}{'...' if len(available) > 5 else ''}")

print("\n" + "="*80)
print("4. DATA QUALITY VERIFICATION:")
print(f"   Total rows: {master_df.shape[0]:,}")
print(f"   Total columns: {master_df.shape[1]}")
print(f"   Missing values: {master_df.isnull().sum().sum()}")
print(f"   Duplicate rows: {master_df.duplicated().sum()}")
print(f"   Memory usage: {master_df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

print("\n5. NUMERIC FEATURE STATISTICS:")
numeric_cols = master_df.select_dtypes(include=[np.number]).columns
print(f"   Total numeric features: {len(numeric_cols)}")
print(f"\n   Sample statistics:")
print(master_df[['total_price', 'quantity', 'frequency', 'rfm_score', 
                'product_popularity_rank']].describe())

PREPROCESSING SUMMARY:

1. DATA QUALITY FIXES:
   - Fixed column name: coustomer_key -> customer_key
   - Handled all missing values (0 remaining)
   - Cleaned and standardized all dimension tables

2. FEATURE ENGINEERING SUMMARY:
   Original columns: 31
   Total columns after engineering: 86
   New features created: 55

3. FEATURE CATEGORIES:

   Temporal Features: 6 features
      day_of_week, is_weekend, time_of_day, is_peak_hour, days_since_start...

   Customer Features: 7 features
      recency, frequency, monetary_avg, rfm_score, customer_segment...

   Product Features: 5 features
      product_transaction_count, product_popularity_rank, price_category, price_vs_category_avg, discount_indicator

   Geographic Features: 4 features
      store_total_revenue, division_avg_transaction, is_dhaka, is_top_division

   Interaction Features: 3 features
      customer_product_interaction, customer_spending_velocity, product_quantity_ratio

   Encoded Features: 4 features
      desc_encod

In [11]:
print("SAVING PREPROCESSED DATA:")
print("="*80)

output_path = 'data/processed/'
import os
os.makedirs(output_path, exist_ok=True)

master_df.to_csv(f'{output_path}master_dataset.csv', index=False)
print(f"1. Master dataset saved: {output_path}master_dataset.csv")
print(f"   Shape: {master_df.shape}")

fact_sales.to_csv(f'{output_path}fact_sales_clean.csv', index=False)
dim_customer.to_csv(f'{output_path}dim_customer_clean.csv', index=False)
dim_item.to_csv(f'{output_path}dim_item_clean.csv', index=False)
dim_store.to_csv(f'{output_path}dim_store_clean.csv', index=False)
dim_time.to_csv(f'{output_path}dim_time_clean.csv', index=False)
dim_payment.to_csv(f'{output_path}dim_payment_clean.csv', index=False)
print(f"\n2. Clean dimension tables saved")

feature_list = {
    'all_features': list(master_df.columns),
    'numeric_features': list(master_df.select_dtypes(include=[np.number]).columns),
    'categorical_features': list(master_df.select_dtypes(include=['object']).columns),
    'encoded_features': [col for col in master_df.columns if '_encoded' in col],
    'onehot_features': [col for col in master_df.columns if any(prefix in col for prefix in ['payment_', 'time_', 'segment_'])]
}

import json
with open(f'{output_path}feature_list.json', 'w') as f:
    json.dump(feature_list, f, indent=4, default=str)
print(f"\n3. Feature list saved: {output_path}feature_list.json")

print("\n" + "="*80)
print("PREPROCESSING COMPLETE")
print("="*80)
print("\nKey Deliverables:")
print("1. Master dataset with 86 features (1M rows)")
print("2. Clean dimension tables with no missing values")
print("3. Feature engineering: temporal, customer, product, geographic, interaction")
print("4. Categorical encoding: label encoding + one-hot encoding")
print("5. Ready for modeling and analysis")

print("\nNext Steps:")
print("- Notebook 03: Demand and Revenue Forecasting")
print("- Notebook 04: Pricing and Optimization")
print("- Notebook 05: Customer Analytics and Segmentation")

SAVING PREPROCESSED DATA:
1. Master dataset saved: data/processed/master_dataset.csv
   Shape: (1000000, 86)

2. Clean dimension tables saved

3. Feature list saved: data/processed/feature_list.json

PREPROCESSING COMPLETE

Key Deliverables:
1. Master dataset with 86 features (1M rows)
2. Clean dimension tables with no missing values
3. Feature engineering: temporal, customer, product, geographic, interaction
4. Categorical encoding: label encoding + one-hot encoding
5. Ready for modeling and analysis

Next Steps:
- Notebook 03: Demand and Revenue Forecasting
- Notebook 04: Pricing and Optimization
- Notebook 05: Customer Analytics and Segmentation
